# 🛠️ Built-in Tools: Extending LLM Capabilities

In this notebook, you'll learn how to use OpenAI's built-in tools that extend Large Language Models beyond text generation, giving them the ability to search the web, access files, execute code, and more.

---

## 🎯 What You'll Learn

By the end of this notebook, you'll be able to:

1. **Understand the Tools ecosystem** - What built-in tools are available and when to use each
2. **Use Web Search** - Enable LLMs to search the internet for up-to-date information
3. **Connect to Remote MCP Servers** - Leverage the Model Context Protocol for external integrations
4. **Implement File Search** - Query your own documents using vector-based search (RAG)
5. **Use Code Interpreter** - Let the LLM write and execute Python code for data analysis
6. **Understand Computer Use** - Learn about the cutting-edge capability for GUI automation

---

## 📋 Prerequisites

**Important:** This notebook builds upon concepts from the Function Calling notebook (Notebook 11). While function calling is a type of tool, it allows you to define **custom functions** that the LLM can call. This notebook focuses on **built-in tools** provided by OpenAI that require no custom implementation.

If you haven't completed the Function Calling notebook, we recommend doing so first to understand:
- How tools work in general
- The request-response cycle
- Tool schemas and parameters

---

## 🔧 Built-in Tools Overview

OpenAI provides several built-in tools that you can enable with just a few lines of code:

| Tool | Purpose | Use Case |
|------|---------|----------|
| **Web Search** | Search the internet | Current events, real-time data, fact-checking |
| **Remote MCP** | Connect to external services | Third-party integrations, external APIs |
| **File Search** | Query uploaded documents | Knowledge bases, documentation, RAG |
| **Code Interpreter** | Execute Python code | Data analysis, calculations, visualizations |
| **Computer Use** | Control computer interfaces | GUI automation, web browsing (advanced) |

Unlike function calling where you define and implement functions yourself, these tools are **hosted by OpenAI** - you simply enable them and the model handles the rest.

---

Let's get started! 🚀

---

# 🔧 Setup

Before we dive into the tools, let's set up our environment.

## 📦 Install Dependencies

In [ ]:
# Install the OpenAI library
!pip install -q openai

# Suppress deprecation warnings for cleaner output
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

print("✅ All dependencies installed!")

## 🔑 API Key Configuration

You have two methods to provide your OpenAI API key:

**Method 1 (Recommended)**: Use Colab Secrets
1. Click the 🔑 icon in the left sidebar
2. Click "Add new secret"
3. Name: `OPENAI_API_KEY`
4. Value: Your OpenAI API key
5. Enable notebook access

**Method 2 (Fallback)**: Manual input when prompted

In [ ]:
# Configure OpenAI API key
# Method 1: Try to get API key from Colab secrets (recommended)
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Method 2: Manual input (fallback)
    from getpass import getpass
    print("💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: OPENAI_API_KEY")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Set the API key as an environment variable
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Validate that the API key is set
if not OPENAI_API_KEY or OPENAI_API_KEY.strip() == "":
    raise ValueError("❌ ERROR: No API key provided!")

print("✅ Authentication configured!")

## 🚀 Initialize OpenAI Client

In [ ]:
# Import required libraries
from openai import OpenAI
import json

# Initialize the OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Configure which OpenAI model to use
OPENAI_MODEL = "gpt-5-nano"  # Cost-efficient model with tool support
# Note: Some tools like Computer Use require specific models (e.g., "computer-use-preview")

print("✅ OpenAI client initialized successfully!")
print(f"🤖 Selected Model: {OPENAI_MODEL}")
print("\n🎓 Ready to explore built-in tools!")

---

# 🌐 1. Web Search

## What is Web Search?

The **Web Search** tool allows the LLM to search the internet for up-to-date information. This is incredibly useful because:

- LLMs have a **knowledge cutoff date** - they don't know about recent events
- Real-time information (weather, stock prices, news) requires live data
- Fact-checking and verification often need current sources

## How It Works

1. You enable web search by adding `{"type": "web_search_preview"}` to the tools array
2. When the model needs current information, it automatically searches the web
3. Search results are incorporated into the model's response
4. The model cites its sources in the response

## 💡 Key Points

- **Automatic**: The model decides when to search based on the query
- **Cited**: Responses include source URLs for verification
- **Current**: Access to information beyond the training data cutoff

---

## 🧪 Example: Basic Web Search

In [ ]:
# Enable web search and ask about current events
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[{"type": "web_search_preview"}],  # Enable web search
    input="What are the latest developments in AI this week?"
)

print("🌐 Web Search Results:")
print("=" * 60)
print(response.output_text)
print("=" * 60)

## 🧪 Example: Real-Time Information

In [ ]:
# Ask about something that requires real-time data
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[{"type": "web_search_preview"}],
    input="What is the current weather in Tokyo, Japan?"
)

print("🌤️ Weather Query Results:")
print("=" * 60)
print(response.output_text)
print("=" * 60)

## 🧪 Example: Fact-Checking with Sources

In [ ]:
# Ask a question that requires verified information
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[{"type": "web_search_preview"}],
    input="Who won the most recent Nobel Prize in Physics and for what discovery?"
)

print("🏆 Nobel Prize Query:")
print("=" * 60)
print(response.output_text)
print("=" * 60)

## 🔍 Inspecting Web Search Results

The response object contains detailed information about the web search, including the sources used. Let's examine the structure:

In [ ]:
# Make a query and inspect the full response
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[{"type": "web_search_preview"}],
    input="What are the top 3 most popular programming languages in 2025?"
)

print("📊 Response Analysis:")
print("=" * 60)
print(f"\n📝 Main Response:")
print(response.output_text)

# Check for web search results in the output
print(f"\n🔍 Output Items:")
for i, item in enumerate(response.output):
    print(f"  {i+1}. Type: {item.type}")

print("=" * 60)

## 🧪 Example: Combining Web Search with Conversation Context

In [ ]:
# Use web search with a multi-turn conversation
messages = [
    {
        "role": "system",
        "content": "You are a helpful research assistant. Use web search to find accurate, current information. Always cite your sources."
    },
    {
        "role": "user",
        "content": "I'm researching electric vehicles. What are the best-selling EVs globally right now?"
    }
]

response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[{"type": "web_search_preview"}],
    input=messages
)

print("🚗 EV Research Results:")
print("=" * 60)
print(response.output_text)
print("=" * 60)

## 📊 Web Search: When to Use It

| ✅ Good Use Cases | ❌ Not Ideal For |
|-------------------|------------------|
| Current events and news | Historical facts (model already knows) |
| Real-time data (weather, stocks) | Creative writing |
| Recent product releases | General knowledge questions |
| Fact-checking claims | Private/internal information |
| Research with citations | Tasks requiring no external data |

## 💰 Pricing Note

Web search incurs additional costs beyond standard token pricing. Each search query has an associated cost, so use it judiciously for queries that truly need current information.

---

# 🔌 2. Remote MCP (Model Context Protocol)

## What is MCP?

The **Model Context Protocol (MCP)** is an open standard introduced by Anthropic in November 2024 and adopted by OpenAI in March 2025. It standardizes how AI systems integrate with external tools, systems, and data sources.

Think of MCP as a **universal adapter** - instead of building custom integrations for every external service, MCP provides a standard interface that any AI model can use to communicate with any MCP-compatible server.

## How Remote MCP Works

```
┌─────────────┐     ┌─────────────────┐     ┌──────────────────┐
│   Your App  │────▶│   OpenAI API    │────▶│  Remote MCP      │
│             │     │   (Responses)   │     │  Server          │
└─────────────┘     └─────────────────┘     └──────────────────┘
                           │                        │
                           │   Tools are listed     │
                           │   and called           │
                           │   automatically        │
                           ▼                        ▼
                    Model decides          External service
                    when to use tools      executes actions
```

1. You specify an MCP server URL in the tools array
2. OpenAI's runtime connects to the server and discovers available tools
3. The model can then call these tools as needed
4. Results are returned and incorporated into the response

## 💡 Key Benefits

- **No Custom Code**: OpenAI handles the tool calling automatically
- **Standardized**: Works with any MCP-compatible server
- **Secure**: Authentication handled via headers
- **Scalable**: Connect to multiple MCP servers simultaneously

---

## 🌐 Using the Fetch MCP Server

For this demo, we'll use the **Fetch MCP Server** - a free, public MCP server that can fetch and process web content. This server:
- Retrieves content from URLs
- Converts HTML to markdown for easier processing
- Requires no API key (beyond your OpenAI key)

### Example: Fetching Web Content

In [ ]:
# Using the Fetch MCP server to retrieve web content
# This is a public MCP server that fetches URLs and converts HTML to markdown

response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[
        {
            "type": "mcp",
            "server_label": "fetch",
            "server_url": "https://mcp.fetch.tools/sse",
            "require_approval": "never"  # Auto-approve tool calls for this demo
        }
    ],
    input="Fetch the content from https://example.com and summarize what you find."
)

print("🔌 MCP Fetch Results:")
print("=" * 60)
print(response.output_text)
print("=" * 60)

## 🔍 Understanding MCP Response Structure

When using MCP, the response includes information about which tools were discovered and used:

In [ ]:
# Make an MCP request and examine the response structure
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[
        {
            "type": "mcp",
            "server_label": "fetch",
            "server_url": "https://mcp.fetch.tools/sse",
            "require_approval": "never"
        }
    ],
    input="Fetch https://httpbin.org/json and tell me what data it contains."
)

print("📊 MCP Response Analysis:")
print("=" * 60)

# Examine each item in the output
for i, item in enumerate(response.output):
    print(f"\n📦 Output Item {i+1}:")
    print(f"   Type: {item.type}")

    # Check for MCP-specific items
    if item.type == "mcp_list_tools":
        print(f"   Server: {item.server_label}")
        print(f"   Tools discovered: {len(item.tools)}")
        for tool in item.tools:
            print(f"     - {tool.get('name', 'unnamed')}")
    elif item.type == "mcp_call":
        print(f"   Tool called: {item.name}")
    elif item.type == "message":
        print(f"   Content preview: {item.content[0].text[:100]}...")

print("\n" + "=" * 60)
print("\n📝 Final Response:")
print(response.output_text)

## 🧪 Example: Fetching Documentation

In [ ]:
# Use MCP to fetch and analyze documentation
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[
        {
            "type": "mcp",
            "server_label": "fetch",
            "server_url": "https://mcp.fetch.tools/sse",
            "require_approval": "never"
        }
    ],
    input="Fetch the Python requests library documentation from https://requests.readthedocs.io/en/latest/ and give me a quick overview of its main features."
)

print("📚 Documentation Summary:")
print("=" * 60)
print(response.output_text)
print("=" * 60)

## 🏢 MCP in Production

While we used the free Fetch server for this demo, in production you can connect to many other MCP servers:

| MCP Server | Purpose | Requires API Key |
|------------|---------|------------------|
| Fetch | Web content retrieval | No |
| GitMCP | GitHub repository access | No (public repos) |
| Zapier | Workflow automation | Yes (Zapier) |
| Stripe | Payment processing | Yes (Stripe) |
| Shopify | E-commerce data | Yes (Shopify) |
| Twilio | Messaging/SMS | Yes (Twilio) |

### Adding Authentication

For servers that require authentication, you can pass headers:

```python
{
    "type": "mcp",
    "server_label": "my_service",
    "server_url": "https://mcp.myservice.com/sse",
    "headers": {
        "Authorization": "Bearer YOUR_API_KEY"
    },
    "require_approval": "never"
}
```

## 💰 Pricing Note

There is **no additional cost** to call remote MCP servers through OpenAI - you're simply billed for the output tokens as usual.

---

# 📁 3. File Search (Vector Store RAG)

## What is File Search?

**File Search** is OpenAI's built-in Retrieval-Augmented Generation (RAG) solution. It allows you to:

1. Upload your own documents (PDFs, text files, etc.)
2. OpenAI automatically chunks, embeds, and indexes them
3. Query your documents using natural language
4. Get answers grounded in your specific content

This is essentially **hosted RAG** - OpenAI handles all the complexity of:
- Text chunking
- Embedding generation
- Vector storage
- Similarity search

## How It Works

```
┌─────────────┐     ┌─────────────────┐     ┌──────────────────┐
│  Your Files │────▶│  Vector Store   │────▶│  File Search     │
│  (upload)   │     │  (auto-indexed) │     │  Tool            │
└─────────────┘     └─────────────────┘     └──────────────────┘
                           │                        │
                           │   Chunks stored        │  User query
                           │   as embeddings        │  searches
                           ▼                        ▼
                    Automatic               Relevant chunks
                    processing              added to context
```

## 💡 Key Benefits

- **No Infrastructure**: No need to set up your own vector database
- **Automatic Processing**: Files are chunked and embedded automatically
- **Simple API**: Just upload files and query
- **Scalable**: Handles large document collections

---

## 📤 Step 1: Upload Files

First, we need to upload our demo files. For this tutorial, we have three files about a fictional company "TechStart Inc.":
- `company_policies.txt` - HR policies and guidelines
- `product_catalog.txt` - Product descriptions and pricing
- `faq.txt` - Frequently asked questions

**Note:** You'll need to upload these files to Google Colab first. Use the file browser on the left sidebar to upload the files from the `demo_files` folder.

In [ ]:
# First, let's upload our demo files to Colab
# Run this cell and select the three .txt files from the demo_files folder

from google.colab import files

print("📤 Please upload the demo files:")
print("   - company_policies.txt")
print("   - product_catalog.txt")
print("   - faq.txt")
print("\nClick 'Choose Files' below to select them...\n")

uploaded = files.upload()

print(f"\n✅ Uploaded {len(uploaded)} file(s):")
for filename in uploaded.keys():
    print(f"   - {filename}")

## 📤 Step 2: Upload Files to OpenAI

Now we upload the files to OpenAI's storage. This makes them available for indexing.

In [ ]:
# Upload files to OpenAI
file_paths = ["company_policies.txt", "product_catalog.txt", "faq.txt"]
uploaded_files = []

print("📤 Uploading files to OpenAI...")
print("=" * 60)

for file_path in file_paths:
    try:
        with open(file_path, "rb") as f:
            file = client.files.create(
                file=f,
                purpose="assistants"  # Purpose for file search
            )
            uploaded_files.append(file)
            print(f"✅ Uploaded: {file_path}")
            print(f"   File ID: {file.id}")
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        print("   Please upload the demo files first (run the previous cell)")

print("=" * 60)
print(f"\n📁 Total files uploaded: {len(uploaded_files)}")

## 🗄️ Step 3: Create a Vector Store

A Vector Store is a container that holds your indexed files. When you add files to a vector store, OpenAI automatically:
1. Chunks the text into smaller pieces
2. Creates embeddings for each chunk
3. Stores everything for efficient retrieval

In [ ]:
# Create a vector store
print("🗄️ Creating Vector Store...")

vector_store = client.vector_stores.create(
    name="TechStart Knowledge Base"
)

print(f"✅ Vector Store created!")
print(f"   ID: {vector_store.id}")
print(f"   Name: {vector_store.name}")
print(f"   Status: {vector_store.status}")

## 📎 Step 4: Add Files to the Vector Store

Now we attach our uploaded files to the vector store for indexing.

In [ ]:
import time

# Add files to the vector store
print("📎 Adding files to Vector Store...")
print("=" * 60)

for file in uploaded_files:
    client.vector_stores.files.create(
        vector_store_id=vector_store.id,
        file_id=file.id
    )
    print(f"   Added: {file.filename}")

# Wait for processing to complete
print("\n⏳ Waiting for files to be processed...")

while True:
    vs_status = client.vector_stores.retrieve(vector_store.id)
    file_counts = vs_status.file_counts

    print(f"   Status: {file_counts.completed}/{file_counts.total} files processed", end="\r")

    if file_counts.completed == file_counts.total:
        break

    time.sleep(1)

print("\n" + "=" * 60)
print(f"\n✅ All files processed and indexed!")
print(f"   Total chunks: {vs_status.usage.total_count if hasattr(vs_status.usage, 'total_count') else 'N/A'}")

## 🔍 Step 5: Query Your Documents

Now comes the exciting part - querying your documents using natural language!

In [ ]:
# Store the vector store ID for use in queries
VECTOR_STORE_ID = vector_store.id

def search_documents(query):
    """Search documents using the file search tool."""
    response = client.responses.create(
        model=OPENAI_MODEL,
        tools=[
            {
                "type": "file_search",
                "vector_store_ids": [VECTOR_STORE_ID]
            }
        ],
        input=query
    )
    return response.output_text

print("✅ Search function ready!")
print(f"📁 Using Vector Store: {VECTOR_STORE_ID}")

## 🧪 Example Queries

Let's test our document search with various questions:

In [ ]:
# Query 1: HR Policy Question
print("📋 Query 1: Vacation Policy")
print("=" * 60)
result = search_documents("How many vacation days do employees get per year?")
print(result)
print("=" * 60)

In [ ]:
# Query 2: Product Information
print("💰 Query 2: Product Pricing")
print("=" * 60)
result = search_documents("What is the pricing for DataFlow Pro?")
print(result)
print("=" * 60)

In [ ]:
# Query 3: Technical Question
print("🔐 Query 3: Security Features")
print("=" * 60)
result = search_documents("What MFA methods are supported by SecureAuth 360?")
print(result)
print("=" * 60)

In [ ]:
# Query 4: FAQ Question
print("❓ Query 4: Account Management")
print("=" * 60)
result = search_documents("How do I reset my password if I forgot it?")
print(result)
print("=" * 60)

In [ ]:
# Query 5: Cross-document query
print("🔄 Query 5: Cross-Document Search")
print("=" * 60)
result = search_documents("What are the remote work requirements and what equipment is provided to remote workers?")
print(result)
print("=" * 60)

## 🧹 Cleanup: Delete Vector Store (Optional)

When you're done, you can delete the vector store to avoid storage charges. Remember, the first 1GB is free, but it's good practice to clean up resources you're not using.

In [ ]:
# Uncomment and run this cell to delete the vector store
# This will delete all indexed data - only do this when you're done!

# client.vector_stores.delete(VECTOR_STORE_ID)
# print(f"🗑️ Vector Store {VECTOR_STORE_ID} deleted!")

print("💡 To delete the vector store, uncomment the lines above and run this cell.")
print(f"   Vector Store ID: {VECTOR_STORE_ID}")

## 💰 File Search Pricing

| Component | Cost |
|-----------|------|
| Vector Storage | First 1GB free, then $0.10/GB/day |
| Search Queries | $2.50 per 1,000 queries |

For a tutorial with small files like ours (< 30KB total), storage is essentially free. You only pay for the queries.

---

# 💻 4. Code Interpreter

## What is Code Interpreter?

**Code Interpreter** allows the LLM to write and execute Python code in a secure, sandboxed environment. This is incredibly powerful for:

- **Data Analysis**: Process CSV files, calculate statistics, find patterns
- **Math & Calculations**: Solve complex equations, perform numerical analysis
- **Visualizations**: Create charts and graphs
- **File Processing**: Transform, clean, and manipulate data
- **Problem Solving**: Write and test code to solve programming challenges

## How It Works

```
┌─────────────┐     ┌─────────────────┐     ┌──────────────────┐
│   User      │────▶│   LLM writes    │────▶│  Sandboxed       │
│   Query     │     │   Python code   │     │  Python Runtime  │
└─────────────┘     └─────────────────┘     └──────────────────┘
                           │                        │
                           │   Code generation      │  Code execution
                           │                        │
                           ▼                        ▼
                    Model refines            Results returned
                    if errors occur          (data, charts, files)
```

## 💡 Key Features

- **Iterative**: If code fails, the model can debug and retry
- **Libraries**: Access to pandas, numpy, matplotlib, and more
- **File Output**: Can generate files (images, CSVs) that you can download
- **Secure**: Code runs in an isolated sandbox

---

## 🧪 Example: Mathematical Calculations

In [ ]:
# Use code interpreter for complex calculations
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[{"type": "code_interpreter"}],
    input="Calculate the first 20 Fibonacci numbers and find their sum. Also calculate the golden ratio approximation using the last two numbers."
)

print("🔢 Mathematical Calculation:")
print("=" * 60)
print(response.output_text)
print("=" * 60)

## 🔍 Inspecting Code Interpreter Output

Let's examine what happens behind the scenes when Code Interpreter runs:

In [ ]:
# Make a request and inspect the code that was executed
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[{"type": "code_interpreter"}],
    input="Generate 100 random numbers between 1 and 1000, then calculate the mean, median, standard deviation, and find the top 5 largest numbers."
)

print("📊 Code Interpreter Analysis:")
print("=" * 60)

# Examine the output items to see the code execution
for i, item in enumerate(response.output):
    print(f"\n📦 Output Item {i+1}: {item.type}")

    if item.type == "code_interpreter_call":
        print(f"\n   📝 Code Executed:")
        print("   " + "-" * 40)
        # The code is in the input field
        if hasattr(item, 'input'):
            for line in item.input.split('\n'):
                print(f"   {line}")
        print("   " + "-" * 40)

        # Show the output/result
        if hasattr(item, 'output'):
            print(f"\n   📤 Output:")
            print(f"   {item.output[:500]}..." if len(str(item.output)) > 500 else f"   {item.output}")

print("\n" + "=" * 60)
print("\n📝 Final Response:")
print(response.output_text)

## 🧪 Example: Data Analysis with Sample Data

In [ ]:
# Create sample sales data for analysis
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[{"type": "code_interpreter"}],
    input="""Create a sample dataset of monthly sales data for a company over 2 years (24 months).
    Include columns: month, sales_amount, expenses, and region (North, South, East, West).
    Then analyze the data to find:
    1. Total sales and expenses per region
    2. Average monthly profit (sales - expenses)
    3. The best and worst performing months
    4. Which region has the highest profit margin"""
)

print("📈 Sales Data Analysis:")
print("=" * 60)
print(response.output_text)
print("=" * 60)

## 🧪 Example: Creating Visualizations

In [ ]:
# Ask code interpreter to create a visualization
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[{"type": "code_interpreter"}],
    input="""Create a visualization showing the growth of major social media platforms from 2010 to 2024.
    Use estimated user numbers (in billions) for Facebook, YouTube, Instagram, TikTok, and Twitter/X.
    Create a line chart with clear labels and a legend."""
)

print("📊 Visualization Created:")
print("=" * 60)
print(response.output_text)

# Check if there are any file outputs (images)
for item in response.output:
    if hasattr(item, 'type') and 'file' in item.type.lower():
        print(f"\n🖼️ Generated file: {item}")

## 🧪 Example: Solving Programming Problems

In [ ]:
# Ask code interpreter to solve a programming challenge
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[{"type": "code_interpreter"}],
    input="""Write a Python function to check if a number is a prime number.
    Then use it to find all prime numbers between 1 and 100.
    Finally, calculate what percentage of numbers from 1-100 are prime."""
)

print("🔢 Prime Number Analysis:")
print("=" * 60)
print(response.output_text)
print("=" * 60)

## 🧪 Example: Working with Uploaded Files

Code Interpreter can also process files you upload. Let's create a sample CSV and analyze it:

In [ ]:
# First, create a sample CSV file locally
import csv
import random

# Generate sample employee data
departments = ['Engineering', 'Sales', 'Marketing', 'HR', 'Finance']
data = []

for i in range(50):
    data.append({
        'employee_id': f'EMP{1000 + i}',
        'department': random.choice(departments),
        'salary': random.randint(50000, 150000),
        'years_experience': random.randint(1, 20),
        'performance_score': round(random.uniform(2.0, 5.0), 1)
    })

# Write to CSV
with open('employee_data.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print("✅ Created employee_data.csv with 50 records")
print("\nPreview:")
for row in data[:5]:
    print(f"  {row}")

In [ ]:
# Upload the CSV to OpenAI
with open('employee_data.csv', 'rb') as f:
    csv_file = client.files.create(
        file=f,
        purpose='assistants'
    )

print(f"✅ Uploaded CSV file")
print(f"   File ID: {csv_file.id}")

In [ ]:
# Use code interpreter to analyze the uploaded file
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[
        {
            "type": "code_interpreter",
            "file_ids": [csv_file.id]  # Attach the file to code interpreter
        }
    ],
    input="""Analyze the employee data CSV file. Please provide:
    1. Basic statistics (count, mean, median) for salary and years_experience
    2. Average salary by department
    3. Correlation between years_experience and salary
    4. Top 5 highest paid employees
    5. Which department has the highest average performance score?"""
)

print("📊 Employee Data Analysis:")
print("=" * 60)
print(response.output_text)
print("=" * 60)

## 💰 Code Interpreter Pricing

| Component | Cost |
|-----------|------|
| Container Session | $0.03 per session |
| Session Duration | Active for 1 hour, 30-minute idle timeout |

Each request that uses Code Interpreter starts a container session. The session remains active for subsequent requests within the timeout period.

---

# 🖥️ 5. Computer Use (Preview)

## What is Computer Use?

**Computer Use** (also called Computer-Using Agent or CUA) is OpenAI's most advanced tool that allows AI models to interact with graphical user interfaces (GUIs) - just like a human would.

The model can:
- View screenshots of a computer screen
- Control mouse movements and clicks
- Type on a virtual keyboard
- Navigate websites, fill forms, and interact with applications

## How It Works

```
┌─────────────┐     ┌─────────────────┐     ┌──────────────────┐
│  Screenshot │────▶│   CUA Model     │────▶│  Action Output   │
│  (pixels)   │     │   (analyzes)    │     │  (click, type)   │
└─────────────┘     └─────────────────┘     └──────────────────┘
       ▲                                           │
       │                                           │
       └───────────────────────────────────────────┘
                    Loop until task complete
```

1. Your application captures a screenshot
2. The screenshot is sent to the CUA model
3. The model analyzes the image and decides what action to take
4. It returns an action (click coordinates, text to type, etc.)
5. Your application executes the action and captures a new screenshot
6. The loop continues until the task is complete

## 💡 Key Capabilities

- **Vision-Based**: Works with any GUI, no special APIs needed
- **Adaptive**: Can handle errors and unexpected situations
- **Multi-Step**: Completes complex, multi-step workflows
- **Universal**: Works on websites, desktop apps, and more

---

## 🔒 Current Limitations

Computer Use is currently in **research preview** with some restrictions:

| Aspect | Details |
|--------|----------|
| **Availability** | Usage tiers 3-5 only |
| **Pricing** | $3/1M input tokens, $12/1M output tokens |
| **Infrastructure** | Requires you to provide the computer environment |
| **Safety** | Recommended to use isolated VMs or containers |

## 📝 API Structure (Reference)

Here's what a Computer Use API call looks like (not executable in this notebook):

```python
# NOTE: This is for reference only - requires special access and infrastructure

response = client.responses.create(
    model="computer-use-preview",  # Special model for computer use
    tools=[
        {
            "type": "computer_use_preview",
            "display_width": 1920,
            "display_height": 1080,
            "environment": "browser"  # or "desktop"
        }
    ],
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Go to google.com and search for 'OpenAI API documentation'"
                },
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": "<base64-encoded-screenshot>"
                    }
                }
            ]
        }
    ]
)

# The response contains actions like:
# - {"type": "click", "x": 500, "y": 300}
# - {"type": "type", "text": "OpenAI API documentation"}
# - {"type": "key", "key": "enter"}
# - {"type": "scroll", "direction": "down", "amount": 3}
```

## 🌟 Use Cases

Computer Use is ideal for:

1. **Browser Automation**: Form filling, web scraping, testing
2. **Desktop Automation**: Interacting with legacy applications
3. **Data Entry**: Automating repetitive data entry tasks
4. **Testing**: End-to-end UI testing without writing test scripts
5. **Personal Assistants**: Booking travel, managing calendars, online shopping

## 🔗 Resources

For more information about Computer Use:

- [OpenAI Computer-Using Agent](https://openai.com/index/computer-using-agent/)
- [OpenAI CUA Sample App](https://github.com/openai/openai-cua-sample-app)
- [Operator (Consumer Product)](https://openai.com/index/introducing-operator/)

---

**Note**: As of July 2025, Operator (the consumer-facing version of Computer Use) has been integrated into ChatGPT as "agent mode", making these capabilities more accessible to end users.

---

# 📚 Summary and Best Practices

## Tool Comparison Matrix

| Tool | Best For | Complexity | Cost |
|------|----------|------------|------|
| **Web Search** | Current events, real-time data | Low | Per query |
| **Remote MCP** | External integrations | Medium | Tokens only |
| **File Search** | Document Q&A, knowledge bases | Medium | Storage + queries |
| **Code Interpreter** | Data analysis, calculations | Low | Per session |
| **Computer Use** | GUI automation, complex workflows | High | Premium pricing |

## Best Practices

### 1. Choose the Right Tool

```
Need current information?      → Web Search
Need external service data?    → Remote MCP
Need to query your documents?  → File Search
Need calculations/analysis?    → Code Interpreter
Need GUI interaction?          → Computer Use
Need custom logic?             → Function Calling (Notebook 11)
```

### 2. Combine Tools When Needed

You can enable multiple tools in a single request:

```python
response = client.responses.create(
    model=OPENAI_MODEL,
    tools=[
        {"type": "web_search_preview"},
        {"type": "code_interpreter"}
    ],
    input="Search for the latest stock price of Apple and calculate the P/E ratio."
)
```

### 3. Consider Costs

- Web Search and Code Interpreter have per-use charges
- File Search has storage costs (first 1GB free)
- Remote MCP is token-cost only
- Computer Use is significantly more expensive


---

## 🔗 Additional Resources

- [OpenAI Tools Documentation](https://platform.openai.com/docs/guides/tools)
- [OpenAI Cookbook](https://cookbook.openai.com/)
- [MCP Tool Guide](https://cookbook.openai.com/examples/mcp/mcp_tool_guide)
- [File Search Guide](https://platform.openai.com/docs/guides/tools-file-search)
- [Code Interpreter Guide](https://platform.openai.com/docs/guides/tools-code-interpreter)

